In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

## 8.
_연습문제: 호크라이터와 슈미트후버는 LSTM에 관한 [논문](https://homl.info/93)에서 임베딩된 레버 문법을 사용했습니다. 이는 ‘BPBTSXXVPSEPE’와 같은 문자열을 만드는 인공 문법입니다. 이 주제에 대한 제니 오어의 훌륭한 소개(https://homl.info/108)를 확인해보세요. 특정 임베딩된 레버 문법 하나를 선택하고(제니 오어의 페이지에 있는 것과 같은), 그다음에 문자열이 이 문법을 따르는지 아닌지 구별하는 RNN을 훈련해보세요. 먼저 문법에 맞는 문자열 50%와 그렇지 않은 문자열 50%를 담은 훈련 배치를 생성하는 함수를 만들어야 합니다._

In [5]:
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output = []
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [6]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(default_reber_grammar), end=" ")

BTXXTTVPXTVPXTTVPSE BPVPSE BTXSE BPVVE BPVVE BTSXSE BPTVPXTTTVVE BPVVE BTXSE BTXXVPSE BPTTTTTTTTVVE BTXSE BPVPSE BTXSE BPTVPSE BTXXTVPSE BPVVE BPVVE BPVVE BPTTVVE BPVVE BPVVE BTXXVVE BTXXVVE BTXXVPXVVE 

In [7]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(embedded_reber_grammar), end=" ")

BTBPTTTVPXTVPXTTVPSETE BPBPTVPSEPE BPBPVVEPE BPBPVPXVVEPE BPBTXXTTTTVVEPE BPBPVPSEPE BPBTXXVPSEPE BPBTSSSSSSSXSEPE BTBPVVETE BPBTXXVVEPE BPBTXXVPSEPE BTBTXXVVETE BPBPVVEPE BPBPVVEPE BPBTSXSEPE BPBPVVEPE BPBPTVPSEPE BPBTXXVVEPE BTBPTVPXVVETE BTBPVVETE BTBTSSSSSSSXXVVETE BPBTSSSXXTTTTVPSEPE BTBPTTVVETE BPBTXXTVVEPE BTBTXSETE 

In [12]:
POSSIBLE_CHARS = "BEPSTVX"

def generate_corrupted_string(grammer, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammer)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char))) # change charactor

    return good_string[:index] + bad_char + good_string[index+1 :]

In [13]:
np.random.seed(42)

for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

BTBPTTTPPXTVPXTTVPSETE BPBTXEEPE BPBPTVVVEPE BPBTSSSSXSETE BPTTXSEPE BTBPVPXTTTTTTEVETE BPBTXXSVEPE BSBPTTVPSETE BPBXVVEPE BEBTXSETE BPBPVPSXPE BTBPVVVETE BPBTSXSETE BPBPTTTPTTTTTVPSEPE BTBTXXTTSTVPSETE BBBTXSETE BPBTPXSEPE BPBPVPXTTTTVPXTVPXVPXTTTVVEVE BTBXXXTVPSETE BEBTSSSSSXXVPXTVVETE BTBXTTVVETE BPBTXSTPE BTBTXXTTTVPSBTE BTBTXSETX BTBTSXSSTE 

In [14]:
def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

In [15]:
string_to_ids("BTTTXXVVETE")

[0, 4, 4, 4, 6, 6, 5, 5, 1, 4, 1]

In [16]:
# dataset with 50% right, 50% wrong

def generate_dataset(size):
    good_strings = [string_to_ids(generate_string(embedded_reber_grammar))
                    for _ in range(size // 2)] # list in a list
    bad_strings = [string_to_ids(generate_corrupted_string(embedded_reber_grammar))
                   for _ in range(size // 2)]
    all_strings = good_strings + bad_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1) # TENSOR WITH NON-UNIFORM RANK
    y = np.array([[1.0] for _ in range(len(good_strings))] + [[0.0] for _ in range(len(bad_strings))])
    
    return X, y

In [17]:
np.random.seed(42)

X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)

2022-07-28 19:16:53.904984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 19:16:53.929450: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 19:16:53.929628: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 19:16:53.930255: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [18]:
X_train[0]

<tf.Tensor: shape=(22,), dtype=int32, numpy=
array([0, 4, 0, 2, 4, 4, 4, 5, 2, 6, 4, 5, 2, 6, 4, 4, 5, 2, 3, 1, 4, 1],
      dtype=int32)>

In [20]:
y_train[-1]

array([0.])

In [21]:
embedding_size = 20

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True), # use ragged=True to use ragged tensors
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
    keras.layers.LSTM(50),
    keras.layers.Dense(1, activation="sigmoid"),
])
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100


/home/dongho/.local/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/lstm/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/lstm/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 20), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/lstm/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


 17/313 [>.............................] - ETA: 0s - loss: 0.6929 - accuracy: 0.4926  

2022-07-28 19:24:53.926879: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302


313/313 [==============================] - 3s 4ms/step - loss: 0.6933 - accuracy: 0.5022 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 2/100
313/313 [==============================] - 1s 4ms/step - loss: 0.6929 - accuracy: 0.5003 - val_loss: 0.6921 - val_accuracy: 0.5000
Epoch 3/100
313/313 [==============================] - 2s 5ms/step - loss: 0.6916 - accuracy: 0.5224 - val_loss: 0.6908 - val_accuracy: 0.4715
Epoch 4/100
313/313 [==============================] - 1s 3ms/step - loss: 0.6898 - accuracy: 0.5308 - val_loss: 0.6890 - val_accuracy: 0.5640
Epoch 5/100
313/313 [==============================] - 1s 3ms/step - loss: 0.6856 - accuracy: 0.5544 - val_loss: 0.6843 - val_accuracy: 0.4650
Epoch 6/100
313/313 [==============================] - 1s 4ms/step - loss: 0.6790 - accuracy: 0.5678 - val_loss: 0.6768 - val_accuracy: 0.6110
Epoch 7/100
313/313 [==============================] - 2s 5ms/step - loss: 0.6690 - accuracy: 0.5873 - val_loss: 0.6686 - val_accuracy: 0.5420
Epoch 8/100

Epoch 58/100
313/313 [==============================] - 1s 4ms/step - loss: 7.1931e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 59/100
313/313 [==============================] - 2s 5ms/step - loss: 6.8293e-04 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 60/100
313/313 [==============================] - 1s 4ms/step - loss: 6.4553e-04 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 61/100
313/313 [==============================] - 2s 7ms/step - loss: 6.1528e-04 - accuracy: 1.0000 - val_loss: 9.8996e-04 - val_accuracy: 1.0000
Epoch 62/100
313/313 [==============================] - 1s 3ms/step - loss: 5.8738e-04 - accuracy: 1.0000 - val_loss: 9.6190e-04 - val_accuracy: 1.0000
Epoch 63/100
313/313 [==============================] - 1s 4ms/step - loss: 5.6139e-04 - accuracy: 1.0000 - val_loss: 9.6931e-04 - val_accuracy: 0.9995
Epoch 64/100
313/313 [==============================] - 1s 3ms/step - loss: 5.4024e-04 - accuracy: 1

In [22]:
test_strings = ["BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
                "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"]
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)
y_proba = model.predict(X_test)

print()
print("레버 문자열일 추정 확률:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))


레버 문자열일 추정 확률:
BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE: 0.73%
BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE: 99.99%


## 9.
_연습문제: 날짜 문자열 포맷을 변환하는 인코더-디코더 모델을 훈련하세요(예를 들어, ‘April 22, 2019’에서 ‘2019-04-22’로 바꿉니다)._

In [23]:
from datetime import date
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()
    
    ordinals = np.random.randint(max_date - min_date, size = n_dates)+ min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]
    
    x = [MONTHS[dt.month -1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    
    return x, y


In [24]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


In [27]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, "))) ### use set to ignore duplicate!!

In [28]:
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

In [29]:
OUTPUT_CHARS = "0123456789-"

In [33]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [34]:
date_str_to_ids(x_example[0], INPUT_CHARS)

[19, 23, 31, 34, 23, 28, 21, 23, 32, 0, 4, 2, 1, 0, 9, 2, 9, 7]

In [35]:
date_str_to_ids(y_example[0], OUTPUT_CHARS)

[7, 0, 7, 5, 10, 0, 9, 10, 2, 0]

In [36]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # use 0 as padding token!
def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [37]:
np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [38]:
Y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1], dtype=int32)>

### 첫 번째 버전: 기본적인 seq2seq 모델
먼저 가장 간단한 모델을 시도해 보겠습니다: 입력 시퀀스가 먼저 (임베딩 층 뒤에 하나의 LSTM 층으로 구성된) 인코더를 통과하여 벡터로 출력됩니다. 그 다음 이 벡터가 (하나의 LSTM 층 뒤에 밀집 층으로 구성된) 디코더로 들어가 벡터의 시퀀스를 출력합니다. 각 벡터는 가능한 모든 출력 문자에 대한 추정 확률입니다.

디코더는 시퀀스를 입력으로 기대하기 때문에 가능한 가장 긴 출력 시퀀스만큼 (인코더의 출력) 벡터를 반복합니다.

In [42]:
embedding_size = 32
max_output_length = Y_train.shape[1]

encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1, 
                           output_dim = embedding_size, 
                           input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True), #return sequence!
    keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")
])

model = keras.models.Sequential([
    encoder, 
    keras.layers.RepeatVector(max_output_length),# just repeat vector!
    decoder
])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, Y_train, epochs=50, validation_data=(X_valid, Y_valid))

Epoch 1/50
313/313 [==============================] - 4s 6ms/step - loss: 1.8045 - accuracy: 0.3551 - val_loss: 1.4062 - val_accuracy: 0.4743
Epoch 2/50
313/313 [==============================] - 1s 4ms/step - loss: 1.2313 - accuracy: 0.5484 - val_loss: 1.0980 - val_accuracy: 0.5980
Epoch 3/50
313/313 [==============================] - 1s 4ms/step - loss: 1.1665 - accuracy: 0.5862 - val_loss: 0.9322 - val_accuracy: 0.6582
Epoch 4/50
313/313 [==============================] - 1s 4ms/step - loss: 1.1515 - accuracy: 0.5898 - val_loss: 0.9523 - val_accuracy: 0.6572
Epoch 5/50
313/313 [==============================] - 1s 4ms/step - loss: 0.8271 - accuracy: 0.6939 - val_loss: 0.6718 - val_accuracy: 0.7370
Epoch 6/50
313/313 [==============================] - 1s 4ms/step - loss: 0.5898 - accuracy: 0.7658 - val_loss: 0.5229 - val_accuracy: 0.7843
Epoch 7/50
313/313 [==============================] - 2s 6ms/step - loss: 0.5686 - accuracy: 0.7851 - val_loss: 0.4258 - val_accuracy: 0.8276
Epoch 

In [44]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("*"+chars)[index] for index in sequence])
            for sequence in ids] #"*" because output is added by 1
                # join inputs a list of string!

In [45]:
X_new = prepare_date_strs(["September 17, 2009", "July 14, 1789"])

In [46]:
ids = np.argmax(model.predict(X_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2009-09-17
1789-07-14


하지만 (가장 긴 날짜에 해당하는) 길이가 18인 입력 문자열에서만 모델이 훈련되었기 때문에 짧은 시퀀스에서는 잘 동작하지 않습니다:

In [47]:
X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"])

In [48]:
ids = np.argmax(model.predict(X_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2020-02-02
1789-09-14


이런! 패딩을 사용해 훈련할 때와 동일한 길이의 시퀀스를 전달해야 할 것 같습니다. 이를 위해 헬퍼 함수를 작성해 보죠:

In [49]:
max_input_length = X_train.shape[1]
def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]]) # tf.pad
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = np.argmax(model.predict(X), axis=-1)
    return ids_to_date_strs(ids)

In [50]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

['2020-05-02', '1789-07-14']

### 두 번째 버전: 디코더에서 쉬프트된 타깃 주입하기(티처 포싱(teacher forcing))
디코더에세 인코더 출력 벡터를 단순히 반복한 것을 주입하는 대신 한 타임 스텝 오른쪽으로 이동된 타깃 시퀀스를 주입할 수 있습니다. 이렇게 하면 각 타임 스텝에서 디코더는 이전 타깃 문자가 무엇인지 알게 됩니다. 이는 더 복잡한 시퀀스-투-시퀀스 문제를 다루는데 도움이 됩니다.

각 타깃 시퀀스의 첫 번째 출력 문자는 이전 문자가 없기 때문에 시퀀스 시작(start-of-sequence, sos)을 나타내는 새로운 토큰이 필요합니다.

추론에서는 타깃을 알지 못하므로 디코더에게 무엇을 주입해야 할까요? sos 토큰을 시작해서 한 번에 하나의 문자를 예측하고 디코더에게 지금까지 예측한 모든 문자를 주입할 수 있습니다(나중에 이 노트북에서 더 자세히 알아 보겠습니다).

하지만 디코더의 LSTM이 스텝마다 이전 타깃을 입력으로 기대한다면 인코더의 벡터 출력을 어떻게 전달할까요? 한가지 방법은 출력 벡터를 무시하는 것입니다. 그리고 대신 인코더의 LSTM 상태를 디코더의 LSTM의 초기 상태로 사용합니다(이렇게 하려면 인코더의 LSTM과 디코더의 LSTM 유닛 개수가 같아야 합니다).

그럼 (훈련, 검증, 테스트를 위한) 디코더의 입력을 만들어 보죠. sos 토큰은 가능한 출력 문자의 마지막 ID + 1으로 나타냅니다.

In [51]:
sos_id = len(OUTPUT_CHARS) + 1
def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value = sos_id) # an array
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1) # disgard last axis of Y!

X_train_decoder = shifted_output_sequences(Y_train) # input the target!!
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

In [52]:
X_train_decoder

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[12,  8,  1, ..., 10, 11,  3],
       [12,  9,  6, ...,  6, 11,  2],
       [12,  8,  2, ...,  2, 11,  2],
       ...,
       [12, 10,  8, ...,  2, 11,  4],
       [12,  2,  2, ...,  3, 11,  3],
       [12,  8,  9, ...,  8, 11,  3]], dtype=int32)>

In [53]:
# use functional API to cerate model!

In [56]:
encoder_embedding_size = 64
decoder_embedding_size = 64
lstm_units = 256

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = keras.layers.Embedding(
    input_dim=len(INPUT_CHARS)+1,
    output_dim=encoder_embedding_size)(encoder_input)

_, encoder_state_h, encoder_state_c = keras.layers.LSTM(
    lstm_units, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = keras.layers.Embedding(
    input_dim=len(OUTPUT_CHARS)+2, 
    output_dim = decoder_embedding_size)(decoder_input)

decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(
    decoder_embedding, initial_state=encoder_state) # feed initial-state like this!
decoder_output = keras.layers.Dense(len(OUTPUT_CHARS)+1, activation="softmax")(
    decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input], 
                           outputs=[decoder_output])


In [57]:
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [58]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=20,
                    validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/20
313/313 [==============================] - 4s 6ms/step - loss: 1.5512 - accuracy: 0.4213 - val_loss: 1.1221 - val_accuracy: 0.5710
Epoch 2/20
313/313 [==============================] - 2s 6ms/step - loss: 0.6676 - accuracy: 0.7566 - val_loss: 0.2324 - val_accuracy: 0.9341
Epoch 3/20
313/313 [==============================] - 2s 8ms/step - loss: 0.1189 - accuracy: 0.9737 - val_loss: 0.0361 - val_accuracy: 0.9973
Epoch 4/20
313/313 [==============================] - 2s 6ms/step - loss: 0.0303 - accuracy: 0.9965 - val_loss: 0.0103 - val_accuracy: 0.9999
Epoch 5/20
313/313 [==============================] - 3s 10ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.0050 - val_accuracy: 0.9999
Epoch 6/20
313/313 [==============================] - 2s 5ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0031 - val_accuracy: 0.9999
Epoch 7/20
313/313 [==============================] - 2s 6ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch

In [59]:
# 이 모델을 사용해 몇 가지 예측을 수행해 보죠. 이번에는 한 문자씩 예측해야 합니다.

In [60]:
sos_id = len(OUTPUT_CHARS) + 1
def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=(len(X),1), value = sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - Y_pred.shape[1]
        X_decoder = tf.pad(Y_pred, [[0,0], [0,pad_size]]) # make partial input
        Y_probas_next = model.predict([X, X_decoder])[:, index:index+1]
        Y_pred_next =tf.argmax(Y_probas_next, axis=-1, output_type=tf.int32)
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis=-1)
    return ids_to_date_strs(Y_pred[:, 1:]) # disgard the first output because it's sos token!


In [61]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

### 세 번째 버전: TF-Addons의 seq2seq 구현 사용하기
정확히 동일한 모델을 만들어 보죠. 하지만 TF-Addon의 seq2seq API를 사용하겠습니다. 아래 구현은 이 노트북의 위에 있는 TFA 예제와 거의 비슷합니다. 다만 모델 입력에 출력 시퀀스 길이를 지정하지 않습니다(하지만 출력 시퀀스의 길이가 매우 다른 프로젝트에서 필요하다면 쉽게 이를 추가할 수 있습니다).

In [63]:
import tensorflow_addons as tfa

encoder_embedding_size = 64
decoder_embedding_size= 64
units = 256

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[],dtype=np.int32)

encoder_embeddings = keras.layers.Embedding(
    len(INPUT_CHARS)+1, encoder_embedding_size)(encoder_inputs)
decoder_embedding_layer = keras.layers.Embedding(
    len(OUTPUT_CHARS)+2, decoder_embedding_size) #중복!!
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True) #중복!!
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()# make a sampler for training!

decoder_cell = keras.layers.LSTMCell(units) # lstm cell
output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell,
                                                 sampler,
                                                 output_layer=output_layer)

final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, 
    initial_state = encoder_state)

Y_proba = keras.layers.Activation("softmax")(final_outputs.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs, decoder_inputs],
                           outputs=[Y_proba])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [64]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=30,
                    validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/30
313/313 [==============================] - 6s 13ms/step - loss: 1.6187 - accuracy: 0.3915 - val_loss: 1.3750 - val_accuracy: 0.4697
Epoch 2/30
313/313 [==============================] - 4s 11ms/step - loss: 0.9696 - accuracy: 0.6242 - val_loss: 0.5014 - val_accuracy: 0.8268
Epoch 3/30
313/313 [==============================] - 4s 11ms/step - loss: 0.2300 - accuracy: 0.9341 - val_loss: 0.0539 - val_accuracy: 0.9944
Epoch 4/30
313/313 [==============================] - 4s 11ms/step - loss: 0.0467 - accuracy: 0.9933 - val_loss: 0.0168 - val_accuracy: 0.9995
Epoch 5/30
313/313 [==============================] - 4s 11ms/step - loss: 0.0091 - accuracy: 0.9999 - val_loss: 0.0067 - val_accuracy: 1.0000
Epoch 6/30
313/313 [==============================] - 4s 11ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 7/30
313/313 [==============================] - 4s 11ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000

In [65]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

In [66]:
inference_sampler = tfa.seq2seq.sampler.GreedyEmbeddingSampler(
    embedding_fn=decoder_embedding_layer)
inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
    decoder_cell, inference_sampler, output_layer=output_layer,
    maximum_iterations=max_output_length) # only make layer
batch_size = tf.shape(encoder_inputs)[:1]
start_tokens = tf.fill(dims=batch_size, value=sos_id)
final_outputs, final_state, final_sequence_lengths = inference_decoder(
    start_tokens,
    initial_state=encoder_state,
    start_tokens=start_tokens,
    end_token=0)

inference_model = keras.models.Model(inputs=[encoder_inputs],
                                     outputs=[final_outputs.sample_id])

몇 개의 노트:
* `GreedyEmbeddingSampler`는 `start_tokens`(디코더 시퀀스마다 sos ID를 담은 벡터)와 `end_token`(모델이 이 토큰을 출력할 때 디코더가 시퀀스 디코딩을 멈춥니다)이 필요합니다.
* `BasicDecoder`를 만들 때 `maximum_iterations`를 설정해야 합니다. 그렇지 않으면 무한하게 반복할 수 있습니다(적어도 하나의 시퀀스에서 모델이 `end_token`을 출력하지 않는다면). 이렇게 되면 주피터 커널을 재시작해야 합니다.
* 모든 디코더 입력이 이전 타임 스텝의 출력을 기반으로 동적으로 생성되기 때문에 디코더 입력은 더 이상 필요하지 않습니다.
* 모델의 출력은 `final_outputs.rnn_outputs`의 소프트맥스가 아니라 `final_outputs.sample_id`입니다. 로짓 값을 얻고 싶다면 `final_outputs.sample_id`을 `final_outputs.rnn_outputs`으로 바꾸세요.

In [68]:
def fast_predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = inference_model.predict(X)
    return ids_to_date_strs(Y_pred)

In [70]:
fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])
### 컴파일 안하면 원래 학습된 모델 재사용 가능!!

['1789-07-14', '2020-05-01']

In [71]:
%timeit predict_date_strs(["July 14, 1789", "May 01, 2020"])

296 ms ± 958 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
%timeit fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])

29.2 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 네 번째 버전: 스케줄 샘플러를 사용하는 TF-Addons의 seq2seq 구현
이전 모델을 훈련할 때 매 타임 스텝 _t_에서 타임 스텝 _t_-1의 타깃 토큰을 모델에게 전달합니다. 하지만 추론에서는 모델이 타임 스텝마다 이전 타깃을 얻을 수 없습니다. 대신에 이전 예측을 사용합니다. 따라서 이런 훈련과 추론 사이에 차이가 실망스러운 성능으로 이어질 수 있습니다. 이를 완화하기 위해 훈련하는 동안 타깃을 예측으로 점진적으로 바꿀 수 있습니다. 이렇게 하려면 `TrainingSampler`를 `ScheduledEmbeddingTrainingSampler`를 바꾸기만 하면 됩니다. 그리고 `sampling_probability`(디코더가 이전 타임 스텝의 타깃 대신에 이전 타임 스텝의 예측을 사용할 확률)를 점진적으로 증가시키기 위해 케라스 콜백을 사용합니다.

In [74]:
sampler = tfa.seq2seq.sampler.ScheduledEmbeddingTrainingSampler(
    sampling_probability=0.0,
    embedding_fn = decoder_embedding_layer)
sampler.sampling_probability= tf.Variable(0.)


decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell,
                                                 sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings,
    initial_state=encoder_state)
Y_proba = keras.layers.Activation("softmax")(final_outputs.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs, decoder_inputs],
                           outputs=[Y_proba])
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [76]:
n_epochs = 50
def update_sampling_probability(epoch, logs):
    proba = min(1.0, epoch / (n_epochs - 10))
    sampler.sampling_probability.assign(proba)
    pass

sampling_probability_cb = keras.callbacks.LambdaCallback(
    on_epoch_begin=update_sampling_probability) # set lambda callback like this!

history = model.fit([X_train, X_train_decoder], Y_train, epochs=n_epochs,
                    validation_data=([X_valid, X_valid_decoder], Y_valid),
                    callbacks=[sampling_probability_cb],
                   )

Epoch 1/50


/home/dongho/.local/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/basic_decoder_2/decoder/while/gradients/model_3/basic_decoder_2/decoder/while/cond_1_grad/Identity_4:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_3/basic_decoder_2/decoder/while/gradients/model_3/basic_decoder_2/decoder/while/cond_1_grad/Identity_3:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model_3/basic_decoder_2/decoder/while/gradients/model_3/basic_decoder_2/decoder/while/cond_1_grad/Identity_5:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/dongho/.local/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/basic_decoder_2/decoder/wh

313/313 [==============================] - 8s 19ms/step - loss: 0.0075 - accuracy: 0.9983 - val_loss: 1.8496e-04 - val_accuracy: 1.0000
Epoch 2/50
313/313 [==============================] - 6s 18ms/step - loss: 9.2833e-05 - accuracy: 1.0000 - val_loss: 1.0697e-04 - val_accuracy: 1.0000
Epoch 3/50
313/313 [==============================] - 6s 18ms/step - loss: 0.0491 - accuracy: 0.9872 - val_loss: 0.1107 - val_accuracy: 0.9694
Epoch 4/50
313/313 [==============================] - 6s 18ms/step - loss: 0.0270 - accuracy: 0.9960 - val_loss: 0.0051 - val_accuracy: 0.9997
Epoch 5/50
313/313 [==============================] - 6s 18ms/step - loss: 0.0053 - accuracy: 0.9995 - val_loss: 0.0042 - val_accuracy: 0.9997
Epoch 6/50
313/313 [==============================] - 6s 18ms/step - loss: 8.4628e-04 - accuracy: 1.0000 - val_loss: 5.0196e-04 - val_accuracy: 1.0000
Epoch 7/50
313/313 [==============================] - 6s 18ms/step - loss: 0.0251 - accuracy: 0.9959 - val_loss: 0.0105 - val_accurac

추론에서도 `GreedyEmbeddingSampler`를 사용해 앞에서와 동일한 작업을 수행할 수 있습니다. 하지만 완성도를 높이기 위해 `SampleEmbeddingSampler`를 사용하겠습니다. 토큰 ID를 찾기 위해 모델 출력에 argmax를 적용하는 대신 로짓 출력에서 랜덤하게 토큰 ID를 샘플링하는 것만 다르고 거의 동일합니다. 텍스트를 생성하는 작업에 유용합니다. `softmax_temperature` 매개변수는 세익스피어와 같은 텍스트를 생성했을 때와 같은 목적을 가집니다(이 매개변수 값이 높을수록 더 랜덤한 텍스트가 생성됩니다).

In [90]:
softmax_temperature = tf.Variable(1.)

inferance_sampler = tfa.seq2seq.sampler.SampleEmbeddingSampler(
    embedding_fn=decoder_embedding_layer,
    softmax_temperature=softmax_temperature)

inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
    decoder_cell, inference_sampler, output_layer=output_layer,
    maximum_iterations=max_output_length)
batch_size = tf.shape(encoder_inputs)[:1] # 동적으로 할당!!
start_tokens = tf.fill(dims=batch_size, value=sos_id)
final_outputs, final_state, final_sequence_lengths = inference_decoder(
    start_tokens,
    initial_state=encoder_state,
    start_tokens=start_tokens,
    end_token=0)

inference_model = keras.models.Model(inputs=[encoder_inputs],
                                     outputs=[final_outputs.sample_id])

In [91]:
def creative_predict_date_strs(date_strs, temperature=1.0):
    softmax_temperature.assign(temperature) # assign value to tf.var
    X = prepare_date_strs_padded(date_strs)
    Y_pred = inference_model.predict(X)
    return ids_to_date_strs(Y_pred)

In [92]:
creative_predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

In [93]:
creative_predict_date_strs(["July 14, 1789", "May 01, 2020"],
                           temperature=5.)

['1789-07-14', '2020-05-01']

### 다섯 번째 버전: TFA seq2seq, 케라스 서브클래싱 API, 어텐션 메커니즘 사용하기
이 문제의 시퀀스는 꽤 짧지만 긴 시퀀스를 처리하려면 어텐션 메커니즘을 사용해야 할 것입니다. 직접 어텐션 메커니즘을 구현할 수 있지만 TF-Addons에 있는 구현을 사용하는 것이 더 간단하고 효율적입니다. 케라스 서브클래싱 API를 사용해서 만들어 보죠.

**경고**: 텐서플로 버그([이슈](https://github.com/tensorflow/addons/issues/1153) 참조) 때문에 즉시 실행 모드(eager mode)에서 `get_initial_state()` 메서드가 실패합니다. 따라서 지금은 `call()` 메서드에서 `tf.function()`을 자동으로 호출하는 (따라서 그래프 모드로 실행하는) 케라스 서브클래싱 API를 사용해야 합니다.

이 구현에서는 간단하게 만들기 위해 다시 `TrainingSampler`를 사용합니다(하지만 `ScheduledEmbeddingTrainingSampler`를 사용해 쉽게 바꿀 수 있습니다). 추론에는 `GreedyEmbeddingSampler`를 사용합니다:

In [111]:
class DateTranslation(keras.models.Model):
    def __init__(self, units=128, encoder_embedding_size=32,
                 decoder_embedding_size=32, **kwargs):
        super().__init__(**kwargs)
        self.encoder_embedding = keras.layers.Embedding(
            input_dim=len(INPUT_CHARS) + 1,
            output_dim=encoder_embedding_size)
        self.encoder = keras.layers.LSTM(units,
                                         return_sequences=True,
                                         return_state=True)
        self.decoder_embedding = keras.layers.Embedding(
            input_dim=len(OUTPUT_CHARS) + 2,
            output_dim=decoder_embedding_size)
        self.attention = tfa.seq2seq.LuongAttention(units)
        decoder_inner_cell = keras.layers.LSTMCell(units)
        self.decoder_cell = tfa.seq2seq.AttentionWrapper(
            cell=decoder_inner_cell,
            attention_mechanism=self.attention)
        output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)
        self.decoder = tfa.seq2seq.BasicDecoder(
            cell=self.decoder_cell,
            sampler=tfa.seq2seq.sampler.TrainingSampler(),
            output_layer=output_layer)
        self.inference_decoder = tfa.seq2seq.BasicDecoder(
            cell=self.decoder_cell,
            sampler=tfa.seq2seq.sampler.GreedyEmbeddingSampler(
                embedding_fn=self.decoder_embedding),
            output_layer=output_layer,
            maximum_iterations=max_output_length)

    def call(self, inputs, training=None):
        encoder_input, decoder_input = inputs
        encoder_embeddings = self.encoder_embedding(encoder_input)
        encoder_outputs, encoder_state_h, encoder_state_c = self.encoder(
            encoder_embeddings,
            training=training)
        encoder_state = [encoder_state_h, encoder_state_c]

        self.attention(encoder_outputs,
                       setup_memory=True)
        
        decoder_embeddings = self.decoder_embedding(decoder_input)

        decoder_initial_state = self.decoder_cell.get_initial_state(
            decoder_embeddings)
        decoder_initial_state = decoder_initial_state.clone(
            cell_state=encoder_state)
        
        if training:
            decoder_outputs, _, _ = self.decoder(
                decoder_embeddings,
                initial_state=decoder_initial_state,
                training=training)
        else:
            start_tokens = tf.zeros_like(encoder_input[:, 0]) + sos_id
            decoder_outputs, _, _ = self.inference_decoder(
                decoder_embeddings,
                initial_state=decoder_initial_state,
                start_tokens=start_tokens,
                end_token=0)

        return tf.nn.softmax(decoder_outputs.rnn_output)

In [113]:
model = DateTranslation()
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit([X_train, X_train_decoder], Y_train, epochs=50,
                     validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/50


2022-07-30 13:26:09.008230: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce GTX 1070" frequency: 1784 num_cores: 15 environment { key: "architecture" value: "6.1" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 2097152 shared_memory_size_per_multiprocessor: 98304 memory_size: 7768375296 bandwidth: 256256000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


312/313 [============================>.] - ETA: 0s - loss: 2.1431 - accuracy: 0.2309

2022-07-30 13:26:15.093913: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce GTX 1070" frequency: 1784 num_cores: 15 environment { key: "architecture" value: "6.1" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 2097152 shared_memory_size_per_multiprocessor: 98304 memory_size: 7768375296 bandwidth: 256256000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


313/313 [==============================] - 9s 20ms/step - loss: 2.1431 - accuracy: 0.2309 - val_loss: 2.1167 - val_accuracy: 0.2285
Epoch 2/50
313/313 [==============================] - 6s 18ms/step - loss: 1.7654 - accuracy: 0.3641 - val_loss: 2.2690 - val_accuracy: 0.2212
Epoch 3/50
313/313 [==============================] - 6s 18ms/step - loss: 1.4464 - accuracy: 0.4766 - val_loss: 1.2961 - val_accuracy: 0.5399
Epoch 4/50
313/313 [==============================] - 6s 18ms/step - loss: 1.3513 - accuracy: 0.5112 - val_loss: 1.2234 - val_accuracy: 0.5457
Epoch 5/50
313/313 [==============================] - 6s 18ms/step - loss: 1.1866 - accuracy: 0.5488 - val_loss: 1.1684 - val_accuracy: 0.5486
Epoch 6/50
313/313 [==============================] - 6s 18ms/step - loss: 1.1640 - accuracy: 0.5502 - val_loss: 1.1550 - val_accuracy: 0.5498
Epoch 7/50
313/313 [==============================] - 6s 18ms/step - loss: 1.2302 - accuracy: 0.5342 - val_loss: 3.8607 - val_accuracy: 0.1853
Epoch 8/50

In [114]:
def fast_predict_date_strs_v2(date_strs):
    X = prepare_date_strs_padded(date_strs)
    X_decoder = tf.zeros(shape=(len(X), max_output_length), dtype=tf.int32)
    Y_probas = model.predict([X, X_decoder])
    Y_pred = tf.argmax(Y_probas, axis=-1)
    return ids_to_date_strs(Y_pred)

In [115]:
fast_predict_date_strs_v2(["July 14, 1789", "May 01, 2020"])

2022-07-30 13:30:57.782160: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce GTX 1070" frequency: 1784 num_cores: 15 environment { key: "architecture" value: "6.1" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 2097152 shared_memory_size_per_multiprocessor: 98304 memory_size: 7768375296 bandwidth: 256256000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


['1789-07-14', '2020-05-01']

TF-Addons에는 몇 가지 흥미로운 기능이 있습니다:
* 추론에 `BasicDecoder` 대신 `BeamSearchDecoder`를 사용하면 가장 높은 확률의 문자를 출력하는 대신 디코더가 몇 개의 후보 중에서 가장 가능성 있는 시퀀스만 유지합니다(자세한 내용은 책 16장을 참고하세요).
* 입력이나 타깃 시퀀스의 길이가 매우 다르면 마스크를 설정하거나 `sequence_length`를 지정합니다.
* `ScheduledEmbeddingTrainingSampler` 보다 더 유연한 `ScheduledOutputTrainingSampler`을 사용하여 타임 스텝 _t_의 출력을 타임 스텝 _t_+1에 주입하는 방법을 결정합니다. 기본적으로 argmax로 ID를 찾지 않고 임베딩 층에 통과시켜 출력을 셀에 바로 주입합니다. 또는 `next_inputs_fn` 함수를 지정하여 셀 출력을 다음 스텝의 입력으로 변환할 수 있습니다.

*바로 주입하면 임베딩의 확률이 나옴!!

## 11.
_연습문제: 최신 언어 모델 중 하나(예를 들어 BERT)로 세익스피어가 쓴 것 같은 텍스트를 생성해보세요._
최신 언어 모델을 사용하는 가장 간단한 방법은 허깅 페이스의 오픈 소스 라이브러리인 [트랜스포머스](https://huggingface.co/transformers/)를 사용하는 것입니다. 이 라이브러리는 자연어 처리(NLP)를 위한 최신 신경망 구조(BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet 등)와 사전훈련된 모델을 많이 제공합니다. 텐서플로와 파이토치를 지원합니다. 무엇보다도 사용하기 매우 쉽습니다.
먼저 사전훈련된 모델을 로드해 보죠. 이 예제에서 추가적인 언어 모델(입력 임베딩에 연결된 가중치를 가진 선형층)을 위에 얹은 OpenAI의 GPT 모델을 사용하겠습니다. 모델을 임포트하고 사전훈련된 가중치를 로드합니다(약 445MB의 데이터가 `~/.cache/torch/transformers`에 다운로드됩니다):

In [116]:
from transformers import TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt")

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

2022-07-30 13:38:36.620703: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFOpenAIGPTLMHeadModel.

All the layers of TFOpenAIGPTLMHeadModel were initialized from the model checkpoint at openai-gpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOpenAIGPTLMHeadModel for predictions without further training.


In [117]:
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/448k [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [118]:
prompt_text = "This royal throne of kings, this sceptred isle"
encoded_prompt = tokenizer.encode(prompt_text,
                                  add_special_tokens=False,
                                  return_tensors="tf")

In [119]:
encoded_prompt

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187]], dtype=int32)>

In [120]:
num_sequences = 5
length = 40

generated_sequences = model.generate(
    input_ids=encoded_prompt,
    do_sample=True,
    max_length=length + len(encoded_prompt[0]),
    temperature=1.0,
    top_k = 0,
    top_p = 0.9,
    repetition_penalty=1.0,
    num_return_sequences=num_sequences,
)



In [121]:
generated_sequences

<tf.Tensor: shape=(5, 50), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   544,  3478, 13762,   239,   547,  1158,   980,  1120,
         1816,   547,  6818,   485,  2150,   618,   239,   487,   961,
          510,   485,  2150,   246,   618,   620,   525,   606,   635,
          589,  1578,  1557,   481,  6404,   239,   244, 40477,   244,
          525,   544,  1849,   240,   244],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   240,   645,   600,   812,  1168,   510,   481,   638,
          485,  1759,   547, 26478,   793,   500,   481,  2264,   498,
         2820,   240,   620,  1311,   617,   481, 17287,   498,   481,
         2185,   498,   589,   481,  8319,   498,   618,   535,   850,
          240,  1048,   249,   485,  3079],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   509, 11764,   562,  1272,   498,   524, 15583,   240,
          568

In [122]:
for sequence in generated_sequences:
    text = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(text)
    print("-"*80)

this royal throne of kings, this sceptred isle is perfectly legitimate. my father has always known my intention to become king. he wanted me to become a king so that we could all hold onto the throne. " 
 " that is true, "
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle, if they will give me the way to start my labour here in the hall of david, so far from the citadel of the lord of all the glory of king's day, am i to learn
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle was ideal for many of his peers, but unlike all king hordal, a mongol count, wanted his most loyal men by their power, instead of the others he took in this order. the threat
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle. 
 " but, dear friends, " the king added, " a love was born 